In [1]:
import pandas as pd
import tkinter as tk
from datetime import datetime
from pandastable import Table, TableModel
from tkinter import scrolledtext
import serial
import time

In [2]:
studentRecord = pd.read_csv('Class Lists.csv')
electronics = pd.read_csv('PowerElectronics.csv')
control = pd.read_csv('ControlSystem.csv')
system = pd.read_csv('PowerSystem.csv')

In [3]:
def StudentInfo(df, rfid):
    matching_row = df[df['RFID'] == rfid]

    if not matching_row.empty:
        sid_value = str(matching_row.iloc[0]['SID'])
        name_value = str(matching_row.iloc[0]['NAME'])
        print(f"The RFID :  {rfid}  belongs to  {name_value}, SID: {sid_value}")
        return (sid_value, name_value)
    else:
        return (str(00000000), 'I_am a ghost')


In [4]:
def create_new_column(df, column_name):
    """Create a new column in a Pandas DataFrame."""
    df[column_name] = 0


In [5]:
def add_value_to_column_sid_wise(df, column_name, sid_value, value_to_add):
    
    if column_name not in df.columns:
        print("Column does not exist.")
        return

    df.loc[df['SID'] == sid_value, column_name] = value_to_add

In [6]:
def add_to_status(text):
    status.configure(state='normal')
    status.insert(tk.INSERT , text + '\n')
    status.configure(state='disabled')

def runningAtt():
    print('statingATTT')
    add_to_status('Starting attandence')
    try:
        serial_port = 'COM4' 
        baud_rate = 115200
        
        ser = serial.Serial(serial_port, baud_rate)
        line = ''
        
        while (line != 'C3 13 F5 FA'):

            line = ser.readline().decode('utf-8').strip()
            
            if (line == 'C3 13 F5 FA'):
                print('exiting')
                line = ''
                add_to_status('Attendance is complete')
                return
        
            (new_SID , new_NAME) = StudentInfo(studentRecord, line)
            print(f"SID:  {new_SID},  NAME:  {new_NAME}")
            add_to_status(f"SID:  {new_SID},  NAME:  {new_NAME}")
            lcdDisplay = str(new_SID + "*" + new_NAME)
            add_to_status(lcdDisplay)
        

    except serial.SerialException:
        print('Your ESP32 is not connected')
        add_to_status(f"Failed to open {serial_port}. Make sure the ESP32 is connected and the port is correct.")
    except KeyboardInterrupt:
        print("Serial monitoring stopped.")
    finally:
        print('att over')


def subject_selected(showingSubject):
    # showingSubject = sub.get()
    subject_label.config(text="Subject: " + showingSubject)
    print(f"Showing {showingSubject} Attandence Record")
    show_db(showingSubject)

subjectRecords = {
            'CLass_List': studentRecord, 
            'Power Electronics': electronics, 
            'Control System': control, 
            'Power System': system,
        }
def show_db(subject):
    middle.table = Table(middle, dataframe=subjectRecords[subject], showtoolbar=False, showstatusbar=False ,editable=False, height = 550, width = 700)
    middle.table.show()

    
root = tk.Tk()
root.geometry("1200x800")
root.title("RFID Attendance System")

middle = tk.Frame(root)
middle.grid(row=1, column=0, pady=20, padx=20)

header = tk.Frame(root)
header.grid(row=0, column=0, pady=20, padx=20)

heading_label = tk.Label(header, text="Electrical Department", font=("Helvetica", 24))
heading_label.grid()
date_label = tk.Label(header, text="Current Date: ", font=("Helvetica", 12))
date_label.grid()
time_label = tk.Label(header, text="Current Time: ", font=("Helvetica", 12))
time_label.grid()


top_R = tk.Frame(root)
top_R.grid(row=0, column=1, pady=20, padx=20)


sub = tk.StringVar()
subjectOptions = ['CLass_List', 'Power Electronics', 'Control System', 'Power System']
curr_sub = tk.OptionMenu(top_R, sub, *subjectOptions, command = subject_selected)
subject_label = tk.Label(top_R, text="Subject :", font=("Helvetica", 12))
# start_att = tk.Button(top_R, text='Start\nattandence', command=runningAtt)
start_att = tk.Button(top_R, text='Start\nattandence', command=lambda: add_to_status('Hello World!'))

curr_sub.grid()
subject_label.grid()


bottom = tk.Frame(root)
bottom.grid(row=1, column=1, pady=20, padx=20)

status = scrolledtext.ScrolledText(bottom, wrap=tk.WORD, width=60, height=25, font=("Times New Roman",12))
status.grid()

start_att.grid(row=0, column=3, padx=70)

def update_time_date_labels():
    current_time = datetime.now().strftime("%H:%M:%S")
    current_date = datetime.now().strftime("%d - %m - %Y")
    time_label.config(text="Time: " + current_time)
    date_label.config(text="Date: " + current_date)
    root.after(1000, update_time_date_labels)
update_time_date_labels()

root.mainloop()


Showing CLass_List Attandence Record
Showing Control System Attandence Record
Showing CLass_List Attandence Record
Showing Control System Attandence Record
Showing Power System Attandence Record
Showing CLass_List Attandence Record
Showing Control System Attandence Record
